# Imports de Llibreries

In [85]:
import numpy as np
import pandas as pd
import json

# Càrrega del dataset

In [86]:
with open("./data/dades_assemblea_clima.json") as file:
    file_contents = file.read()

parsed_json = json.loads(file_contents)

# Exploratory Data Analysis

Les dades que hem obtingut sobre les propostes realitzades al procés de l'Assemblea Ciutadana pel Clima de Catalunya han estat recollides a través de l'api oficial amb GraphQL. Això ha fet que les dades estiguin en format JSON i que, per tant, si volem treballar amb fitxers de format CSV, cal que les transformem.

In [87]:
df = pd.json_normalize(parsed_json) # df conté totes les propostes arrel. "df['node.comments']" conté els comentaris resposta al arrel.

In [88]:
df.info()
df.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1 non-null      object
 1   slug         1 non-null      object
 2   publishedAt  1 non-null      object
 3   stats        1 non-null      object
 4   components   1 non-null      object
dtypes: object(5)
memory usage: 168.0+ bytes


,id,slug,publishedAt,stats,components
0,562,assembleaclima,2023-09-13T13:28:49+02:00,"[{'name': 'results_count', 'value': 0}, {'name...","[{'id': '3827', 'name': {'translation': 'Propo..."


Observem que al convertir el fitxer JSON de les dades a un dataframe amb Pandas no ens mostra una columna per cada atribut, sinó que ho divideix en quatre columnes: 
- `id`: Id del procés participatiu
- `slug`: URI i en aquest cas també, nom del procés participatiu
- `publishedAt`: Data de publicació del procés participatiu
- `stats`: Objecte JSON amb diverses estadístiques del procés, com nombre de propostes totals, nombre de pàgines, nombre de meetings i nombre d'enquestes.
- `components`: Components del procés participatiu, en aquest cas, el component de Proposals.

In [89]:
df_propostes = pd.json_normalize(parsed_json['components'][0]['proposals']['edges'])
df_comments = df_propostes['node.comments']

Dividim el dataset anterior en dos datasets diferents per poder analitzar millor les dades més importants:
- `df_propostes`: Dataset que conté la totalitat de les propostes "arrel" i els atributs de cadascuna.
- `df_comments`: Dataset que conté els comentaris a cadascuna de les propostes, si és que en tenen.

In [90]:
df_propostes.info()
df_propostes.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   node.id                         50 non-null     object 
 1   node.state                      3 non-null      object 
 2   node.author.name                50 non-null     object 
 3   node.title.translation          50 non-null     object 
 4   node.body.translation           50 non-null     object 
 5   node.category.id                45 non-null     object 
 6   node.category.name.translation  45 non-null     object 
 7   node.voteCount                  50 non-null     int64  
 8   node.comments                   50 non-null     object 
 9   node.category                   0 non-null      float64
dtypes: float64(1), int64(1), object(8)
memory usage: 4.0+ KB


,node.id,node.state,node.author.name,node.title.translation,node.body.translation,node.category.id,node.category.name.translation,node.voteCount,node.comments,node.category
0,87446,evaluating,Concepción Moreno Maurel,Participar Assemblea Ciutadana pel Clima de Ca...,"<p>Buenos días, me gustaría participar en la a...",2540,Desplegament de les energies renovables,0,[],NaN
1,87447,evaluating,Isabel Soria Muñoz,Assemblea ciutadana pel clima de Catalunya,<p>M'agradaria participar en aquesta assemblea...,2540,Desplegament de les energies renovables,0,"[{'id': '53292', 'author': {'name': 'JORGE MAN...",NaN
2,87450,None,ramonferre,"Municipi a municipi, empresa a empresa, famíli...",<p>Cada municipi (empresa i família) ha de reb...,2540,Desplegament de les energies renovables,0,"[{'id': '53324', 'author': {'name': 'Jordi San...",NaN


En aquest dataset, les propostes eren tractades com un node cadascuna, amb un identificador únic. D'aquí que els atributs de cada columna comenci amb el prefix "node". També s'observa que existeixen atributs que tenen un sufix "translation". Això és degut a que per accedir a l'atribut que precedeix a "translation", calia obligatòriament seleccionar-ne una de les possibles traduccions.
Observem que cada proposta té els següents atributs:
- `node.id`: Identificador únic de cada proposta.
- `node.state`: Estat en què es troba la proposta.
- `node.author.name`: Nom d'usuari de l'autor de la proposta.
- `node.title.translation`: Títol de la proposta.
- `node.body.translation`: Text de la proposta. Està formatejat en paràgraf HTML.
- `node.category.id`: Identificador únic de la categoria a la que pertany la proposta.
- `node.category.name.translation`: Nom de la categoria a la que pertany la proposta.
- `node.voteCount`: Nombre de vots que ha rebut la proposta.
- `node.comments`: JSON que conté els comentaris que ha rebut cada proposta, si és que n'ha rebut.
- `node.category`: Categoria de la proposta.

In [111]:
df_comments.info()
df_comments.head(3)
df_comments_temp = []
for item in df_comments:
    for thing in item:
        new_item = json.dumps(thing)
        df_comments_temp.append(new_item)

<class 'pandas.core.series.Series'>
RangeIndex: 50 entries, 0 to 49
Series name: node.comments
Non-Null Count  Dtype 
--------------  ----- 
50 non-null     object
dtypes: object(1)
memory usage: 528.0+ bytes
